Este notebook é uma adaptação para uso no ambiente Google Colab do notebook **notebook_01.ipynb** fornecido como material complementar do livro Inteligência Artificial: Uma Abordagem de Aprendizado de Máquina|FACELI, Katti; LORENA, Ana C.; GAMA, João; AL, et. Tendo sido desenvolvido originalmente por: Renato Moraes Silva.

Antes de iniciar este notebook, salve o arquivo do conjunto de dados iris (irs.csv) em um diretório local na sua máquina

In [ ]:
# O código abaixo lê o arquivo csv

import pandas as pd

files = pd.read_csv('./iris.csv')
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

: 

## Introdução

Frequentemente, a visualização dos dados auxilia na interpretação e na análise de como eles estão distribuídos. O Python possui algumas bibliotecas que facilitam o processo de visualização, tais como: `Pandas`, `Matplotlib` e `Seaborn`. Para aprender a usar essas ferramentas, será usada a base de dados Iris. É importante destacar que a base de dados Iris usada neste exercício foi modificada pelos autores por motivos didáticos. A versão original dela pode ser encontrada no seguinte link: <https://archive.ics.uci.edu/ml/datasets/iris>. Usando a versão modificada dessa base de dados, será abordado como fazer a eliminação de atributos irrelevantes e o tratamento de valores faltantes. Também será mostrado como tratar valores redundantes ou inconsistentes e como fazer a normalização dos dados. Depois, será feita a detecção e remoção de *outliers* da base dados. Por fim, será mostrado como fazer a análise da distribuição das classes e da correlação entre os atributos. Ao final deste *notebook*, espera-se que o leitor tenha assimilado as principais etapas necessárias para fazer a análise dos dados e prepará-los para serem usados pelos algoritmos de aprendizado de máquina.

Este *notebook* se refere aos Capítulos **2 (Análise de Dados)** e **3 (Pré-processamento de Dados)** do livro Inteligência Artificial: Uma Abordagem de Aprendizado de Máquina.

---
## Carregando os dados

Primeiro, vamos importar todas as bibliotecas que serão usadas ao longo deste exercício.

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np # importa a biblioteca usada para trabalhar com vetores e matrizes
import pandas as pd # importa a biblioteca usada para trabalhar com dataframes (dados em formato de tabela) e análise de dados

# bibliotecas usadas para geracao de graficos
import seaborn as sns
import matplotlib.pyplot as plt

print('Bibliotecas carregadas com sucesso')

Em seguida, os dados do conjunto de dados iris serão importados.

In [ ]:
#importa o arquivo e guarda em um dataframe do Pandas
df_dataset = pd.read_csv( 'iris.csv', sep=',', index_col=None) 

print('Dados importados com sucesso!')

Agora, vamos dar uma olhada nas 10 primeiras amostras da base de dados.

In [ ]:
# exibe os 10 primeiros elementos do dataframe
display(df_dataset.head(n=10))

  A base de dados contém amostras de flores (linhas) representadas pelos seguintes atributos (colunas): `id_planta`, `comprimento_sepala`, `largura_sepala`, `comprimento_petala`, `largura_petala` e `cidade_origem`. Por fim, temos o atributo `classe` que contém a espécie de cada flor.

O atributo `id_planta` é qualitativo, uma vez que é usado para identificar uma determinada amostra. Apesar dele possuir valores numéricos crescentes, ele exerce apenas a função de identificação e seus valores poderiam ser trocados por outros identificadores não numéricos sem nenhum prejuízo. O atributo `cidade_origem` também é qualitativo. Os atributos `comprimento_sepala`, `largura_sepala`, `comprimento_petala` e `largura_petala` são quantitativos contínuos.

Quanto à escala, os atributos `id_planta` e `cidade_origem` são qualitativos nominais, enquanto os atributos `comprimento_sepala`, `largura_sepala`, `comprimento_petala` e `largura_petala` são quantitativos racionais.

O atributo `classe` é qualitativo nominal e representa espécies de flores. Portanto, o problema em questão é de <b>aprendizado supervisionado</b> $\rightarrow$ <b>classificação</b>.

## Pré-processamento: eliminação de atributos irrelevantes

O objetivo do problema é identificar a espécie de uma flor (`classe`), dados os demais atributos. Neste caso, não é preciso uma análise profunda para observar que os atributos `id_planta` e `cidade_origem` não contribuem para a identificação da classe. Portanto, em uma tarefa de aprendizado de máquina, devemos remover esses atributos, pois são irrelevantes. Em cenários reais, muitas vezes é necessário consultar especialistas para ajudar a identificar quais atributos são irrelevantes.

In [ ]:
# remove as colunas id_planta e cidade_origem
df_dataset = df_dataset.drop(columns=['id_planta','cidade_origem'])

# imprime o dataframe
display(df_dataset.head(n=10))

## Pré-processamento: tratamento de atributos com valores ausentes

Outro passo importante, é verificar se existem atributos com valores ausentes (*NaN*) na base de dados.

In [ ]:
# índices das linhas que contém valores NaN
idxRowNan = pd.isnull(df_dataset).any(1).to_numpy().nonzero()

# imprime apenas as linhas com valoes ausentes
display(df_dataset.iloc[idxRowNan])

Como podemos ver, o atributo `largura_sepala` possui 2 amostras com valores ausentes. Já o atributo `comprimento_petala` possui 1 amostra com valor faltante. 

Existem diversas técnicas para tratar atributos faltantes. Como este problema possui poucos valores ausentes, vamos preencher esses valores com a média dos valores conhecidos da respectiva classe (`Iris-setosa`).

In [ ]:
def trataFaltantes( df_dataset ):
    '''
    Substitui os valores faltantes pela média dos outros valores do mesmo atributo
    de amostras que sejam da mesma classe    
    '''
    
    # seleciona apenas as linhas da base de dados onde a coluna largura_sepala não contém valores nulos
    notNull_ls = df_dataset.loc[ ~pd.isnull(df_dataset['largura_sepala']), :]
    notNull_cp = df_dataset.loc[ ~pd.isnull(df_dataset['comprimento_petala']), :]

    # calcula a media dos valores do atributo largura_sepala que não são nulos e que são da classe Iris-setosa 
    media_ls = notNull_ls[ notNull_ls['classe']=='Iris-setosa' ]['largura_sepala'].mean()
    media_cp = notNull_cp[ notNull_cp['classe']=='Iris-setosa' ]['comprimento_petala'].mean()

    # substitui os valores nulos pela média 
    df_dataset.loc[ pd.isnull(df_dataset['largura_sepala']), 'largura_sepala'] = media_ls
    df_dataset.loc[ pd.isnull(df_dataset['comprimento_petala']), 'comprimento_petala'] = media_cp
    
    return df_dataset

trataFaltantes( df_dataset )
    
# imprime apenas as linhas que antes possuiam valores NaN
print('\nAmostras que possuiam valores faltantes:')
display(df_dataset.iloc[idxRowNan])

## Pré-processamento: tratamento de dados inconsistentes ou redundantes

Outro passo importante, é verificar se existem dados inconsistentes ou redundantes. A forma mais comum de inconsistência é quando há amostras representadas por atributos com todos os valores iguais, mas com classes diferentes. A redundância é dada pela repetição de linhas na base de dados.

A seguir, vamos verificar se existem amostras duplicadas (redundantes) e inconsistentes.

In [ ]:
df_duplicates = df_dataset[ df_dataset.duplicated(subset=['comprimento_sepala','largura_sepala','comprimento_petala','largura_petala'],keep=False)] 

# se houver valores redundantes ou inconsistentes, imprima 
if len(df_duplicates)>0:
    print('\nAmostras redundantes ou inconsistentes:')
    display(df_duplicates)
else:
    print('Não existem valores duplicados')

Como podemos ver, existem algumas amostras redundantes (duplicadas) e outras inconsistentes (amostras iguais, mas com classes distintas). 

Primeiro, serão removidas as amostras redundantes, mantendo na base apenas uma delas.

In [ ]:
def delDuplicatas( df_dataset ):
    '''
    Para cada grupo de amostras duplicadas, mantém uma e apaga as demais
    '''
    
    # remove as amostras duplicadas, mantendo apenas a primeira ocorrencia
    df_dataset = df_dataset.drop_duplicates(keep = 'first')    

    return df_dataset

df_dataset = delDuplicatas( df_dataset )


Após remover as amostras redundantes, é preciso checar se há amostras inconsistentes. 

In [ ]:
# para detectar inconsistências, a rotina abaixo obtém as amostras onde os valores 
# dos atributos continuam duplicados. Neste caso, os atributos serão iguais, mas as classes serão distintas
df_duplicates = df_dataset[ df_dataset.duplicated(subset=['comprimento_sepala','largura_sepala','comprimento_petala','largura_petala'],keep=False)] 

# se tiver valores inconsistentes, imprime 
if len(df_duplicates)>0:
    print('\nAmostras inconsistentes:')
    display(df_duplicates)
else:
    print('Não existem mostras inconsistentes')
    

Podemos ver que existem duas amostras inconsistentes. Nesse caso, como não é possível saber qual delas está correta, as duas serão eliminadas.

In [ ]:
def delInconsistencias( df_dataset ):
    '''
    Remove todas as amostras inconsistentes da base de dados
    '''

    df_dataset = df_dataset.drop_duplicates(subset=['comprimento_sepala','largura_sepala','comprimento_petala','largura_petala'], keep = False)    
  
    return df_dataset

df_dataset = delInconsistencias( df_dataset )

# obtém apenas as amostras onde os valores dos atributos estão duplicados
df_duplicates = df_dataset[ df_dataset.duplicated(subset=['comprimento_sepala','largura_sepala','comprimento_petala','largura_petala'],keep=False)] 

# se tiver valores redundantes ou inconsistentes, imprime 
if len(df_duplicates)>0:
    display(df_duplicates)
else:
    print('Não existem amostras redundantes ou inconsistentes')
    

Agora, vamos gerar algumas estatísticas sobre a base de dados.

A função `describe()` da `Pandas` sumariza as principais estatísticas sobre os dados de um *data frame*, como a média, o desvio padrão, valor máximo, valor mínimo e alguns percentis.

In [ ]:
# apresenta as principais estatísticas da base de dados
df_detalhes = df_dataset.describe()

display(df_detalhes)

## Pré-processamento: normalização dos atributos

Observe que a média do atributo `comprimento_sepala` é bastante superior a média do atributo `largura_petala`. Diante disso, está claro que a escala dos atributos é diferente, o que pode prejudicar alguns métodos de aprendizado de máquina. Portanto, vamos normalizar os valores dos atributos para que fiquem com média igual a zero e desvio padrão igual a um. Usando a biblioteca *scikit-learn*, poderíamos fazer a normalização usando a função [*sklearn.preprocessing.StandardScaler*](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Mas, para exercitar os conceitos aprendidos, vamos criar nossa própria função de normalização. 

In [ ]:
def normalizar(X):
    """
    Normaliza os atributos em X
    
    Esta função retorna uma versao normalizada de X onde o valor da
    média de cada atributo é igual a 0 e desvio padrao é igual a 1. Trata-se de
    um importante passo de pré-processamento quando trabalha-se com 
    métodos de aprendizado de máquina.
    """
    
    m, n = X.shape # m = qtde de objetos e n = qtde de atributos por objeto
    
    # Incializa as variaves de saída
    X_norm = np.random.rand(m,n) # inicializa X_norm com valores aleatórios
    mu = 0 # inicializa a média
    sigma = 1 # inicializa o desvio padrão
     
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0, ddof=1)
    
    for i in range(m):
        X_norm[i,:] = (X[i,:]-mu) / sigma
        
    
    return X_norm, mu, sigma


# coloca os valores dos atributos na variável X
X = df_dataset.iloc[:,0:-1].values

# chama a função para normalizar X
X_norm, mu, sigma = normalizar(X)

df_dataset.iloc[:,0:-1] = X_norm

print('\nPrimeira amostra da base antes da normalização: [%2.4f %2.4f].' %(X[0,0],X[0,1]))
print('\nApós a normalização, espera-se que a primeira amostra seja igual a: [-0.5747 0.1804].')
print('\nPrimeira amostra da base apos normalização: [%2.4f %2.4f].' %(X_norm[0,0],X_norm[0,1]))


Agora que os dados estão normalizados, vamos analisar as informações estatísticas novamente.

In [ ]:
# apresenta as principais estatísticas da base de dados
df_detalhes = df_dataset.describe()

display(df_detalhes.round(8))

Podemos ver acima que a média (*mean*) ficou igual a 0 e o desvio padrão (*std*) igual a 1. 

## Pré-processamento: detecção de *outliers*

Outro passo importante na análise e tratamento dos dados é a detecção de *outliers* (*i.e.*, dados gerados por leituras incorretas, erros de digitação, etc). 

Uma das maneiras mais simples de verificar se os dados contém *outliers* é criar um gráfico box plot de cada atributo. Para isso, podemos usar a função `boxplot` da biblioteca `Pandas`.

In [ ]:
# gera um bloxplot para cada atributo
df_dataset.boxplot(figsize=(15,7))
plt.show()

O box plot está indicando que os atributos `comprimento_sepala` e `largura_sepala` possuem *outliers*, o que pode prejudicar o desempenho de vários métodos de aprendizado de máquina, pois tratam-se de amostras com valores de atributos incorretos. 

Outra forma de analisar se a base de dados contém *outliers* é usar gráficos de dispersão. Podemos plotar gráficos de dispersão de todas as combinações de atributos da base de dados usando a função `scatter_matrix` da `Pandas`.

In [ ]:
pd.plotting.scatter_matrix(df_dataset, figsize=(18,18))

plt.show()

Outra forma de plotar gráficos de dispersão a partir dos _dataframes_ é usando a biblioteca `Seaborn`. Juntamente com essa biblioteca, também é recomendável importar a biblioteca `Matplotlib` para personalizar os gráficos. 

In [ ]:
# matriz de gráficos scatter 
sns.pairplot(df_dataset, hue='classe', height=3.5);

# mostra o gráfico usando a função show() da matplotlib
plt.show()

Observando os gráficos de dispersão, é fácil perceber que existem duas amostras da classe *Iris-virginica* que estão deslocadas no espaço em relação às demais amostras.

Pelos gráficos, os *outliers* parecem ser mais visíveis na combinação dos atributos `comprimento_sepala` e `largura_sepala`. Então, vamos usar a função `lmplot` da biblioteca `Seaborn`para visualizar a combinação desses dois atributos.

In [ ]:
# define o scatter plot
sns.lmplot(x='comprimento_sepala', y='largura_sepala', data=df_dataset, 
           fit_reg=False,  
           hue='classe')

# cria um título para o gráfico
plt.title('Comprimento vs largura da sepala')

# mostra o gráfico
plt.show()

Pelos gráficos vistos até o momento, fica claro que um dos *outliers* possui um alto valor no atributo `largura_sepala`. Já o segundo outlier contém um alto valor no atributo `comprimento_sepala`. 

A bilioteca `Seaborn` permite criar gráficos boxplot agrupados por um determinado atributo, o que facilita a análise dos dados. No exemplo abaixo, criaremos boxplots para cada atributo agrupados pela classe.

In [ ]:
for atributo in df_dataset.columns[:-1]:
    # define a dimensão do gráfico
    plt.figure(figsize=(8,8))

    # cria o boxplot
    sns.boxplot(x="classe", y=atributo, data=df_dataset, whis=1.5)

    # mostra o gráfico
    plt.show()

Os box plots dos atributos mostraram outros *outliers* que não haviam aparecido no primeiro box plot. Portanto, esses novos valores são considerados *outliers* se analisarmos as classes individualmente, mas não são considerados *outliers* se analisarmos a base de dados de forma geral. 

Outro tipo de gráfico que ajuda a detectar *outliers* é o histograma. Portanto, vamos usá-lo para analisar cada atributo.

In [ ]:
for atributo in df_dataset.columns[:-1]:
    
    # cria o histograma
    n, bins, patches = plt.hist(df_dataset[atributo].values,bins=10, color='red', edgecolor='black', linewidth=0.9)

    # cria um título para o gráfico
    plt.title(atributo)

    # mostra o gráfico
    plt.show()

Nos histogramas, os *outliers* mais evidentes estão nos atributos `comprimento_sepala` e `largura_sepala`.

Agora, vamos usar um gráfico de densidade para fazer o mesmo tipo de análise.


Uma das maneiras mais simples de tratar *outliers* é remover aqueles valores que são menores que $Q1 - 1.5 * IQR$ ou maiores que $Q3 + 1.5 * IQR$, onde $Q1$ é o primeiro quartil, $Q3$ é o terceiro quartil e $IQR$ é o intervalo interquartil. O IQR pode ser calculado pela seguinte equação: $IQR = Q3-Q1$. 

Com base nessas informações, vamos usar a função abaixo para remover os *outliers* da base de dados. Usaremos como base o IQR de cada atributo em relação a todos os valores na base de dados, em vez do IQR individual de cada classe.

In [ ]:
def removeOutliers(df_dataset):
    """
    Remove os outliers da base de dados 
    """
    
    for atributo in df_dataset.columns[:-1]:

        # obtem o terceiro e o primeiro quartil. 
        q75, q25 = np.percentile(df_dataset[atributo].values, [75 ,25])
        
        # calcula o IQR
        IQR = q75 - q25

        # remove os outliers com base no valor do IQR
        df_dataset = df_dataset[ (df_dataset[atributo]<=(q75+1.5*IQR)) & (df_dataset[atributo]>=(q25-1.5*IQR)) ]
    
    return df_dataset

# remove os outliers
df_dataset = removeOutliers( df_dataset )

# apresenta as principais estatísticas sobre a base de dados
df_dataset.boxplot(figsize=(15,7))
plt.show()

# matriz de gráficos scatter 
sns.pairplot(df_dataset, hue='classe', height=3.5);

# mostra o gráfico usando a função show() da matplotlib
plt.show()

Depois da remoção, o box plot e os gráficos de dispersão indicam que não há mais nenhum *outlier* na base de dados. 

Com os novos gráficos de dispersão, também é possível perceber que a classe *Iris-setosa* é mais fácil de identificar, pois está mais separada no espaço de atributos. Por outro lado, em várias combinações de atributos, as classes *Iris-versicolor* e *Iris-virginica* se misturam.

**IMPORTANTE:** antes de realizar a remoção de *outliers*, é mandatório analisar cuidadosamente as características das amostras antes de removê-las. Em alguns casos, remover os *outliers* pode ser prejudicial. Além disso, algumas tarefas de aprendizado de máquina são voltadas para a detecção de *outliers* e, portanto, esses dados não podem ser removidos. Adicionalmente, se a base de dados for desbalanceada, a remoção dos *outliers* com base nas estatísticas de toda a base, pode acabar removendo amostras da classe minoritária (aquela que possui menos amostras). Ainda, alguns métodos de classificação, tais como métodos baseados em *ensemble* e métodos baseados em árvores, costumam ser robustos a *outliers*. Diante disso, em alguns problemas, é recomendável remover apenas aqueles *outliers* que são claramente erros de leitura/digitação, isto é, valores que estão fora dos limites aceitáveis para o que é esperado para um determinado atributo (por exemplo, uma pessoa com 500 anos ou um bebê com 300 kg). 

## Pré-processamento: distribuição das classes

Outro passo importante na análise de dados é verificar a distribuição das classes. Para isso, é possível criar um gráfico de barra indicando quantas amostras de cada classe há na base de dados.

In [ ]:
display( df_dataset['classe'].value_counts() )

# cria um gráfico de barras com a frequência de cada classe
sns.countplot(x="classe", data=df_dataset)

# mostra o gráfico
plt.show()

Conforme podemos ver, as classes são balanceadas. Se o número de exemplos em alguma das classes fosse muito superior às demais, teríamos que usar alguma técnica de balanceamento de classes, pois o modelo gerado pela maioira dos métodos de aprendizado supervisionado costuma ser tendencioso para as classes com maior número de amostras. 

## Pré-processamento: correlação entre os atributos

Quando dois atributos possuem valores idênticos ou muito semelhantes para todas as amostras, um deles deve ser eliminado ou eles devem ser combinados. Isso ajuda a diminuir o custo computacional das tarefas de aprendizado e evita que o aprendizado de alguns método seja prejudicado, principalmente os métodos baseados em otimização.

Uma das maneiras mais comuns de analisar a correlação dos dados é através das matrizes de correlação e covariância. Podemos fazer isso usando a biblioteca `Numpy` ou a `Pandas`.

Primeiro, vamos fazer usando a `Numpy`.

In [ ]:
# criando uma matriz X com os valores do data frame
X = df_dataset.iloc[:,:-1].values

# matriz de covariancia
covariance = np.cov(X, rowvar=False)

# matriz de correlação
correlation = np.corrcoef(X, rowvar=False)

print('Matriz de covariância: ')
display(covariance)

print('\n\nMatriz de correlação: ')
display(correlation)

Agora, vamos calcular as matrizes de correlação e covariância usando a `Pandas`.

In [ ]:
# matriz de covariancia
df_covariance = df_dataset.cov()

# matriz de correlação
df_correlation = df_dataset.corr()

print('Matriz de covariância: ')
display(df_covariance)

print('\n\nMatriz de correlação: ')
display(df_correlation)

Podemos ver que os atributos `comprimento_petala` e `largura_petala` possuem alta covariância e alta correlação. Se o problema que estamos analisando tivesse muitos atributos, poderíamos pensar na possibilidade de combinar esses dois atributos. Se a correlação entre dois atributos for igual a 1 ou -1, significa que eles são redundantes e um deles poderia ser eliminado.

Para facilitar a visualização, vamos plotar a matriz de covariância e a de correlação usando mapas de cores.

In [ ]:
# cria um mapa de cores dos valores da covariancia
sns.heatmap(df_covariance, 
        xticklabels=df_correlation.columns,
        yticklabels=df_correlation.columns)

plt.title('Covariancia')
plt.show()

# cria um mapa de cores dos valores da correlação
sns.heatmap(df_correlation, 
        xticklabels=df_correlation.columns,
        yticklabels=df_correlation.columns)

plt.title('Correlacao')
plt.show()

---
## Conclusão

Neste notebook, foram mostradas as principais etapas de visualização, interpretação e pré-processamento dos dados. Foi apresentado como  eliminar atributos  irrelevantes e tratar dados faltantes, redundantes ou inconsistentes. Além disso, foi mostrado como deve ser feita a normalização dos dados e quais os possíveis impactos dessa etapa no desempenho dos métodos de aprendizado. Ainda, foi mostrada uma das técnicas de remoção de outliers, como visualizar a distribuição das classes e como analisar a correlação dos atributos. Para obter maiores detalhes teóricos sobre os conceitos apresentados, consulte os Capítulos 2 (Análise de Dados) e 3 (Pré-processamento de Dados).